<a href="https://colab.research.google.com/github/Mangai2024/Mangai2024/blob/main/Complete_RAG_Pipeline_solar_System__Using_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Step 1 Load PDF
!pip install -U langchain-community pypdf
from langchain.document_loaders import PyPDFLoader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 8.7 MB/s eta 0:00:00


In [ ]:
#Load the PDF
loader= PyPDFLoader("//content/solar_system.pdf")

In [ ]:
#Extract text as documents (one per page)
docs = loader.load()

In [ ]:
docs[0].page_content

'Solar System\nThe Solar System consists of the Sun and the objects that orbit it, including eight planets, dwarf\nplanets, moons, asteroids, and comets. The Sun is the central star and provides the energy needed for\nlife on Earth. The planets orbit the Sun in elliptical paths. The inner planets are Mercury, Venus, Earth,\nand Mars. The outer planets are Jupiter, Saturn, Uranus, and Neptune.'

In [ ]:
#Chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
#Create a text splitter
splitter=RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

In [ ]:
#Split the PDF pages into chunks
chunks =splitter.split_documents(docs)

In [ ]:
print("Total chunks:",  len(chunks))
print("First chunk:\n")
print(chunks[0].page_content)

Total chunks: 1
First chunk:

Solar System
The Solar System consists of the Sun and the objects that orbit it, including eight planets, dwarf
planets, moons, asteroids, and comets. The Sun is the central star and provides the energy needed for
life on Earth. The planets orbit the Sun in elliptical paths. The inner planets are Mercury, Venus, Earth,
and Mars. The outer planets are Jupiter, Saturn, Uranus, and Neptune.


In [ ]:
#Embedding Steps
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 38.1 MB/s eta 0:00:00


In [ ]:
#create embedding(no api key needed)
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Create FAISS index from documents and embeddings, then save it
db = FAISS.from_documents(chunks, embeddings)
db.save_local("faiss_hf_index")

# Now load the local FAISS index
db = FAISS.load_local("faiss_hf_index", embeddings, allow_dangerous_deserialization=True)

In [ ]:
#Retrieval only
query="What are the planets in the Solar System?"
retriever =db.as_retriever(search_kwargs={"k":3})
results = retriever.invoke(query)
for r in results:
  print(r.page_content[:400])
  print("-------------------")

Solar System
The Solar System consists of the Sun and the objects that orbit it, including eight planets, dwarf
planets, moons, asteroids, and comets. The Sun is the central star and provides the energy needed for
life on Earth. The planets orbit the Sun in elliptical paths. The inner planets are Mercury, Venus, Earth,
and Mars. The outer planets are Jupiter, Saturn, Uranus, and Neptune.
-------------------


In [ ]:
#Retrievel and Generation (RAG)
#Load the LLM
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

In [ ]:
model_name="google/flan-t5-small"

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=AutoModelForSeq2SeqLM.from_pretrained(model_name)
llm=pipeline("text2text-generation", model=model, tokenizer=tokenizer)

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
#Retrieve Context
query = "What are the planets in the Solar System?"
retriever =db.as_retriever(search_kwargs={"k":3})
docs =retriever.invoke(query)
context = ""
for d in docs:
  context += d.page_content + "\n\n"

In [ ]:
#Generate answer
prompt = f"Context:\n{context}nQuestion: {query}\nAnswer:"
answer = llm(prompt, max_length=200)[0]["generated_text"]
print(answer)

Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


dwarf planets, moons, asteroids, and comets
